In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask.array as da

import time
import sys

from era5_remap import *

In [2]:
# Start and end dates for selecting data
sdate='1979-01-01'
edate='2020-12-31'

in_varname='mtpr'
out_varname='precip'

In [3]:
files_t2m='/shared/land/ERA5/daily/global/compressed_sfc_fc_daily_*.nc4'

In [4]:
# Open files
compressed = xr.open_mfdataset(files_t2m,chunks={'lgrid': 'auto'})
compressed['time']=pd.to_datetime(compressed['time'].values, format='%Y%m%d')
compressed=compressed.sel(time=slice(sdate,edate))
compressed_dates=compressed['time']

In [5]:
compressed

<xarray.Dataset>
Dimensions:   (time: 15341, lgrid: 141780)
Coordinates:
  * time      (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2020-12-31
Dimensions without coordinates: lgrid
Data variables: (12/43)
    lat       (time, lgrid) float32 dask.array<chunksize=(31, 141780), meta=np.ndarray>
    lon       (time, lgrid) float32 dask.array<chunksize=(31, 141780), meta=np.ndarray>
    cin       (time, lgrid) float32 dask.array<chunksize=(31, 141780), meta=np.ndarray>
    zust      (time, lgrid) float32 dask.array<chunksize=(31, 141780), meta=np.ndarray>
    msror     (time, lgrid) float32 dask.array<chunksize=(31, 141780), meta=np.ndarray>
    mssror    (time, lgrid) float32 dask.array<chunksize=(31, 141780), meta=np.ndarray>
    ...        ...
    flsr      (time, lgrid) float32 dask.array<chunksize=(31, 141780), meta=np.ndarray>
    lat_2     (time, lgrid) float32 dask.array<chunksize=(31, 141780), meta=np.ndarray>
    lon_2     (time, lgrid) float32 dask.array<chunksize=(31, 141780), meta=np.ndarray>
    mcpr      (time, lgrid) float32 dask.array<chunksize=(31, 141780), meta=np.ndarray>
    sp        (time, lgrid) float32 dask.array<chunksize=(31, 141780), meta=np.ndarray>
    tcc       (time, lgrid) float32 dask.array<chunksize=(31, 141780), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 1.8.2 (http://mpimet.mpg.de/...
    history:      Fri Apr 16 08:45:44 2021: cdo -z zip_1 -f nc4 -O copy compr...
    Conventions:  CF-1.6
    description:  ERA5 daily statistics computed from hourly output on N320 r...
    CDO:          Climate Data Operators version 1.8.2 (http://mpimet.mpg.de/...

In [6]:
t_start = time.perf_counter()
uncomp_list=[]
for t in range(len(compressed['time'])):
    uncomp = era5_remap(compressed[in_varname][t,:],"land")
    uncomp_list.append(uncomp)
print ("      era5_remap function:",(time.perf_counter()-t_start),"seconds to uncompress all times")

      era5_remap function: 4361.444650610909 seconds to uncompress all times


In [7]:
del(compressed)
del(uncomp)

In [8]:
ds_t2m=xr.combine_nested(uncomp_list,
                         concat_dim='time').to_dataset(name=out_varname).chunk({'time':100})
ds_t2m['time']=compressed_dates
ds_t2m=ds_t2m.rename({'lat':'latitude','lon':'longitude'})

In [9]:
ds_t2m

<xarray.Dataset>
Dimensions:    (time: 15341, longitude: 1280, latitude: 534)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2020-12-31
  * longitude  (longitude) float64 0.0 0.2812 0.5625 ... 359.2 359.4 359.7
  * latitude   (latitude) float64 -60.0 -59.72 -59.44 ... 89.23 89.51 89.78
Data variables:
    precip     (time, latitude, longitude) float32 dask.array<chunksize=(100, 534, 1280), meta=np.ndarray>

In [10]:
ofile='/shared/ccsm4/kpegion/mlso/era5_'+out_varname+'_daily.'+sdate+'.'+edate+'.nc' 
ds_t2m.to_netcdf(ofile)